In [15]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install re
!{sys.executable} -m pip install gensim
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install cython
!{sys.executable} -m pip install tensorflow

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Could not find a version that satisfies the requirement re (from versions: )
No matching distribution found for re
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is avail

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import string
import json
import re
from gensim import corpora
from pprint import pprint
import gensim
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import random
import datetime

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
TEXTFILE = "AllText.json"
FileJson = json.load(open(TEXTFILE))
RawSentences = {}
for i, sentenceid in enumerate(FileJson):
    RawSentences[int(sentenceid)] = gensim.utils.simple_preprocess(FileJson[sentenceid])
    
print(len(RawSentences))
for k, v in RawSentences.items():
    print(k, v)
    break
    


448016
215458535964672 ['ve', 'said', 'it', 'before', 'and', 'will', 'say', 'it', 'again', 'the', 'mainstream', 'media', 'is', 'out', 'to', 'bring', 'down', 'my', 'administration', 'need', 'you', 'to', 'take', 'the', 'media', 'accountability', 'survey', 'to', 'do', 'your', 'part', 'to', 'fight', 'back', 'against', 'the', 'fake', 'news', 'attacks', 'and', 'deceptions']


In [3]:
Sentences = [Sentence for Sentence in RawSentences.values()]
Sentences.append(['__NOT__IN__VOCAB__'] * 11)
print(Sentences[:5])
model = Word2Vec(sentences=Sentences, min_count=10, window=5, workers=20, size=300)
model.train(Sentences, total_examples=len(Sentences), epochs=20)

[['ve', 'said', 'it', 'before', 'and', 'will', 'say', 'it', 'again', 'the', 'mainstream', 'media', 'is', 'out', 'to', 'bring', 'down', 'my', 'administration', 'need', 'you', 'to', 'take', 'the', 'media', 'accountability', 'survey', 'to', 'do', 'your', 'part', 'to', 'fight', 'back', 'against', 'the', 'fake', 'news', 'attacks', 'and', 'deceptions'], ['if', 'we', 'want', 'to', 'change', 'congress', 'we', 'need', 'to', 'change', 'who', 'we', 'send', 'to', 'congress'], ['happy', 'th', 'of', 'july', 'let', 'celebrate', 'our', 'freedom', 'and', 'independence', 'today', 'and', 'cherish', 'them', 'always', 'freedom', 'is', 'never', 'more', 'than', 'one', 'generation', 'away', 'from', 'extinction', 'we', 'didn', 'pass', 'it', 'to', 'our', 'children', 'in', 'the', 'bloodstream', 'it', 'must', 'be', 'fought', 'for', 'protected', 'and', 'handed', 'on', 'for', 'them', 'to', 'do', 'the', 'same', 'president', 'reagan'], ['attention', 'nv', 'homeowners', 'the', 'new', 'summer', 'electric', 'rates', 'ha

(307628226, 387904220)

In [44]:
LABELSFILE = "Text.csv"
def MakeOneHotLabels(label, ID):
    """
    Makes one hot label. 1 at the sentiment. 
    [1,0,0,0,0] for Very Negative
    [0,0,1,0,0] for Neutral
    [0,0,0,0,1] for Very Positive
    """
#     OneHot = [0,0,0]
    
#     label = int(label)
#     if label>0:
#         return [0,0,1]
#     elif label<0:
#         return [1,0,0]
#     return [0,1,0]
    OneHot = [0,0,0,0,0]
    
    label = int(label)
    OneHot[label+2]+=1 #Initializes at 0.
    LabelGroups[label+2].append(ID)
    return OneHot
 
    
    
Labels = {}
with open(LABELSFILE, 'r') as f:
    File = csv.DictReader(f, delimiter=',')
    for row in File:
        ID = int(row['ID'])        
        if row['ClearMajority'] != '-':
            Labels[ID] = MakeOneHotLabels(row['ClearMajority'], ID)
        elif row['SoftMajority'] != '-':
            Labels[ID] = MakeOneHotLabels(row['SoftMajority'], ID)
        elif row['damon'] != '-':
            Labels[ID] = MakeOneHotLabels(row['damon'], ID)

In [59]:
batchSize = 32
lstmUnits = 128
numClasses = 5
iterations = 3000

In [60]:
# Gettings labels and sentences together.
# Weeding out sentences over 120 words to determine max sequence length. 
print(len(Labels))
numWords = []
LabelsGraph = []
for i in Labels.keys():
    numWords.append(len(RawSentences[i]))
    LabelsGraph.append(Labels[i])

IDs = list(Labels.keys())
for ID in IDs:
    if len(RawSentences[ID]) > 120:
        del Labels[ID]
print(len(Labels), len(RawSentences))
IDs = list(Labels.keys())
random.shuffle(IDs)

TrainIDs = IDs[:432]
TestIDs = IDs[432:]

print()

499
499 448016



In [61]:
"""
Figuring out embedding vectors for tensor graph.
"""
MAXSEQUENCELENGTH = 120
numSentencesLabeled = len(Labels)
WORDEMBEDDINGSIZE = 300
print(numSentencesLabeled)


Embeddings = np.zeros((len(model.wv.vocab), WORDEMBEDDINGSIZE), dtype='float32')

WordsToInd = {}
for i, word in enumerate(model.wv.vocab):
    WordsToInd[word] = i
    Embeddings[i] = model.wv[word]
    
WordIndex = np.array([i for i in WordsToInd.values()])

499


In [62]:
"""
Create Batches
"""
    
def GetWordIndices(ID):
    Array = np.zeros([MAXSEQUENCELENGTH], dtype='int32')
    for j, word in enumerate(RawSentences[ID]):
        if word in WordsToInd:
            Array[j] = WordsToInd[word]
        else:
            Array[j] = WordsToInd.get("__NOT__IN__VOCAB__")
    return Array


def GetTrainBatch():
    TrainLabels = np.zeros([batchSize, numClasses], dtype='int32')
    TrainSet = np.zeros([batchSize, MAXSEQUENCELENGTH], dtype='int32')
    IDsBatched = set()
    Count = 0
    while Count < batchSize:
        SelectID = TrainIDs[random.randint(0, len(TrainIDs)-1)]
        if SelectID not in IDsBatched:
            IDsBatched.add(SelectID)
            TrainSet[Count] = GetWordIndices(SelectID)
            TrainLabels[Count] = Labels[SelectID]
            Count += 1
    return TrainSet, TrainLabels

def GetTestBatch():
    TestLabels = np.zeros([batchSize, numClasses])
    TestSet = np.zeros([batchSize, MAXSEQUENCELENGTH])
    TestIDsCovered = set()
    Count = 0
    while Count < batchSize:
        SelectID = TestIDs[random.randint(0, len(TestIDs)-1)]
        
        if SelectID not in TestIDsCovered:
            TestIDsCovered.add(SelectID)
            TestSet[Count] = GetWordIndices(SelectID)
            TestLabels[Count] = Labels[SelectID]
            Count += 1

    return TestSet, TestLabels

In [63]:
tf.reset_default_graph()
labels = tf.placeholder(tf.float32, [batchSize, numClasses])
inputs = tf.placeholder(tf.int32, [batchSize, MAXSEQUENCELENGTH])

data = tf.Variable(tf.zeros([batchSize, MAXSEQUENCELENGTH, WORDEMBEDDINGSIZE]), dtype='float32')
data = tf.nn.embedding_lookup(Embeddings, inputs)


In [64]:
lstmCell = tf.nn.rnn_cell.LSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.5)
initialState = lstmCell.zero_state(batchSize, dtype='float32') 
value, _ = tf.nn.dynamic_rnn(cell=lstmCell, inputs=data, initial_state=initialState, dtype='float32')
print(value.get_shape)

<bound method Tensor.get_shape of <tf.Tensor 'rnn/transpose_1:0' shape=(32, 120, 128) dtype=float32>>


In [65]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0])-1)
prediction = tf.matmul(last, weight) + bias

In [66]:
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [67]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [68]:
sess = tf.InteractiveSession()
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1662: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [69]:
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
for i in range(iterations):
    #Next Batch of reviews
    nextBatch, nextBatchLabels = GetTrainBatch()
    sess.run(optimizer, {inputs: nextBatch, labels: nextBatchLabels})

    #Write summary to Tensorboard
    if i % 50 == 0:
        summary_train = sess.run(merged, {inputs: nextBatch, labels: nextBatchLabels})
        summary_pb = tf.summary.Summary()
        summary_pb.ParseFromString(summary_train)
        print(summary_pb)
        writer.add_summary(summary_pb, i)

    #Save the network every 10,000 training iterations
    if i % 10000 == 0 and i != 0:
        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
        print("saved to %s" % save_path)


writer.close()

value {
  tag: "Loss"
  simple_value: 3.8537018299102783
}
value {
  tag: "Accuracy"
  simple_value: 0.375
}

value {
  tag: "Loss"
  simple_value: 1.5814251899719238
}
value {
  tag: "Accuracy"
  simple_value: 0.28125
}

value {
  tag: "Loss"
  simple_value: 1.5589122772216797
}
value {
  tag: "Accuracy"
  simple_value: 0.375
}

value {
  tag: "Loss"
  simple_value: 1.6195306777954102
}
value {
  tag: "Accuracy"
  simple_value: 0.28125
}

value {
  tag: "Loss"
  simple_value: 1.310251235961914
}
value {
  tag: "Accuracy"
  simple_value: 0.53125
}

value {
  tag: "Loss"
  simple_value: 1.354491949081421
}
value {
  tag: "Accuracy"
  simple_value: 0.53125
}

value {
  tag: "Loss"
  simple_value: 1.1793259382247925
}
value {
  tag: "Accuracy"
  simple_value: 0.40625
}

value {
  tag: "Loss"
  simple_value: 1.301943302154541
}
value {
  tag: "Accuracy"
  simple_value: 0.375
}

value {
  tag: "Loss"
  simple_value: 1.3069329261779785
}
value {
  tag: "Accuracy"
  simple_value: 0.46875
}

v

In [70]:
iterations = 10
for i in range(iterations):
    nextBatch, nextBatchLabels = GetTestBatch();
    print("Accuracy for this batch:", (sess.run(accuracy, {inputs: nextBatch, labels: nextBatchLabels})) * 100)

Accuracy for this batch: 50.0
Accuracy for this batch: 37.5
Accuracy for this batch: 37.5
Accuracy for this batch: 40.625
Accuracy for this batch: 50.0
Accuracy for this batch: 31.25
Accuracy for this batch: 56.25
Accuracy for this batch: 43.75
Accuracy for this batch: 40.625
Accuracy for this batch: 50.0


In [43]:
sess.close()

In [ ]:
def display_closestwords_tsnescatterplot(model, word):
    
    arr = np.empty((0,300), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.similar_by_word(word, topn=30)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.figure(figsize=(10, 6), dpi=800)
    plt.show()

In [ ]:
#print(model['trump'])
display_closestwords_tsnescatterplot(model, 'trump')
#print(model.similar_by_word('trump', topn=30))
display_closestwords_tsnescatterplot(model, 'kavanaugh')
#model.similar_by_word('court', topn=30)
display_closestwords_tsnescatterplot(model, 'fbi')
#model.similar_by_word('fbi', topn=30)